# netCDF File for All Variables (mean, clim, anom,std)

In [1]:
import os
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib
import datetime
from dateutil.relativedelta import relativedelta


In [2]:
os.chdir('d:/CMAP')
ds_precip = xr.open_dataset('precip.mon.mean.nc') # ds_precip

## cut new clim

In [3]:
# new_clim_ds
new_clim = ds_precip.sel(time=slice('1991-01-01','2020-12-01'))
new_clim

<xarray.Dataset>
Dimensions:  (time: 360, lat: 72, lon: 144)
Coordinates:
  * time     (time) datetime64[ns] 1991-01-01 1991-02-01 ... 2020-12-01
  * lon      (lon) float32 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
  * lat      (lat) float32 88.75 86.25 83.75 81.25 ... -83.75 -86.25 -88.75
Data variables:
    precip   (time, lat, lon) float32 ...

In [4]:
# Unit : mm/day -> mm/month
for i in range(len(new_clim.precip)):
    p_time = str(new_clim.time[i].values)[:10] # '1979-03-01'
    b = datetime.datetime.strptime(p_time,'%Y-%m-%d')
    nums_of_day = ((b + relativedelta(months=i))-datetime.timedelta(days=1)).day
    new_clim.precip[i] *= nums_of_day
precip = new_clim.precip

lon = new_clim.lon
lat = new_clim.lat
time = new_clim.time

In [5]:
### add clim variable / len  30
clim_sel = new_clim['precip'].groupby('time.month').mean('time')
new_clim['new_clim'] = clim_sel
new_clim #(check)

<xarray.Dataset>
Dimensions:   (time: 360, lat: 72, lon: 144, month: 12)
Coordinates:
  * time      (time) datetime64[ns] 1991-01-01 1991-02-01 ... 2020-12-01
  * lon       (lon) float32 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
  * lat       (lat) float32 88.75 86.25 83.75 81.25 ... -83.75 -86.25 -88.75
  * month     (month) int32 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    precip    (time, lat, lon) float32 ...
    new_clim  (month, lat, lon) float32 7.182 8.205 8.07 ... 13.07 13.2 13.6

In [52]:
### case 1 
anom_lst = []

for i in range(1, 13):
    each_mon = new_clim['precip'].sel(time=new_clim['time.month'] == i)  # month
    clim_mon = new_clim.new_clim.sel(month=i)  # clim
    anom_sel = each_mon - clim_mon  # anom
    
    anom_lst.append(anom_sel)

anom_cb = xr.concat(anom_lst, dim='time')
new_clim['new_anom'] = anom_cb


### case 2
# anom_lst = []
# for i in range(1, 13):
#     mon_group = new_clim.sel(time=new_clim['time.month'] == i)  # month
#     anom_sel = mon_group['precip'] - clim_sel.sel(month=i)  # anom
    
#     anom_lst.append(anom_sel)
    
# anom_cb = xr.concat(anom_lst, dim='time')
# new_clim['new_anom'] = anom_cb

### case 3
# anom = new_clim.precip.groupby('time'.month).std(dim = 'time',ddof =1)

c:\Users\ocpc\.conda\envs\cartopy\lib\site-packages\xarray\core\concat.py:500: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


In [54]:
# std_lst = []
std_sel=new_clim.precip.groupby('time.month').std(dim='time',ddof=1)

std_cb = xr.concat(std_sel, dim='month')  
new_clim['new_std'] = std_cb



c:\Users\ocpc\.conda\envs\cartopy\lib\site-packages\xarray\core\concat.py:500: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


In [56]:
new_clim.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
new_clim.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
new_clim.precip.attrs ={'units':'mm/month','long_name':'Average Monthly Rate of Precipitation'}
new_clim.new_clim.attrs ={'units':'mm/month','long_name':'Average Long Term Mean Monthly Rate of Precipitation'}
new_clim.new_anom.attrs ={'units':'mm/month','long_name':'Anomaly Monthly Rate of Precipitation'}
new_clim.new_std.attrs ={'unit':'mm/month','long_name':'Standard Deviation Rate of Precipitation'}

In [57]:
## save
# new_clim.to_netcdf('CMAP_ncfile_all_variables(mean,clim,anom,std)_new.nc')

## cut old clim

In [58]:
# old_clim_ds
old_clim = ds_precip.sel(time=slice('1981-01-01','2010-12-01'))
old_clim

<xarray.Dataset>
Dimensions:  (time: 360, lat: 72, lon: 144)
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2010-12-01
  * lon      (lon) float32 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
  * lat      (lat) float32 88.75 86.25 83.75 81.25 ... -83.75 -86.25 -88.75
Data variables:
    precip   (time, lat, lon) float32 ...

In [59]:
# Unit : mm/day -> mm/month
for i in range(len(old_clim.precip)):
    p_time = str(old_clim.time[i].values)[:10] # '1979-03-01'
    b = datetime.datetime.strptime(p_time,'%Y-%m-%d')
    nums_of_day = ((b + relativedelta(months=i))-datetime.timedelta(days=1)).day
    old_clim.precip[i] *= nums_of_day
precip = old_clim.precip

lon = old_clim.lon
lat = old_clim.lat
time = old_clim.time

In [60]:
### add clim variable / len  30
clim_sel = old_clim['precip'].groupby('time.month').mean('time')
old_clim['old_clim'] = clim_sel
old_clim #(check)

<xarray.Dataset>
Dimensions:   (time: 360, lat: 72, lon: 144, month: 12)
Coordinates:
  * time      (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2010-12-01
  * lon       (lon) float32 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
  * lat       (lat) float32 88.75 86.25 83.75 81.25 ... -83.75 -86.25 -88.75
  * month     (month) int32 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    precip    (time, lat, lon) float32 ...
    old_clim  (month, lat, lon) float32 8.887 9.734 9.558 ... 15.83 15.91 15.97

In [61]:
### anom
anom_lst = []

for i in range(1, 13):
    each_mon = old_clim['precip'].sel(time=old_clim['time.month'] == i)  # month
    clim_mon = old_clim.old_clim.sel(month=i)  # clim
    anom_sel = each_mon - clim_mon  # anom
    
    anom_lst.append(anom_sel)

anom_cb = xr.concat(anom_lst, dim='time')
old_clim['old_anom'] = anom_cb


c:\Users\ocpc\.conda\envs\cartopy\lib\site-packages\xarray\core\concat.py:500: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


In [62]:
### std
std_sel=old_clim.precip.groupby('time.month').std(dim='time',ddof=1)

std_cb = xr.concat(std_sel, dim='month')  
old_clim['old_std'] = std_cb

c:\Users\ocpc\.conda\envs\cartopy\lib\site-packages\xarray\core\concat.py:500: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


In [63]:
old_clim.lon.attrs = {'units':'degree_east','long_name':'Longitude'}
old_clim.lat.attrs = {'units':'degree_north','long_name': 'Latitude'}
old_clim.precip.attrs ={'units':'mm/month','long_name':'Average Monthly Rate of Precipitation'}
old_clim.old_clim.attrs ={'units':'mm/month','long_name':'Average Long Term Mean Monthly Rate of Precipitation'}
old_clim.old_anom.attrs ={'units':'mm/month','long_name':'Anomaly Monthly Rate of Precipitation'}
old_clim.old_std.attrs ={'unit':'mm/month','long_name':'Standard Deviation Rate of Precipitation'}

In [64]:
old_clim

<xarray.Dataset>
Dimensions:   (time: 360, lat: 72, lon: 144, month: 12)
Coordinates:
  * time      (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2010-12-01
  * lon       (lon) float32 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
  * lat       (lat) float32 88.75 86.25 83.75 81.25 ... -83.75 -86.25 -88.75
  * month     (month) int32 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    precip    (time, lat, lon) float32 ...
    old_clim  (month, lat, lon) float32 8.887 9.734 9.558 ... 15.83 15.91 15.97
    old_anom  (time, lat, lon) float32 4.753 3.906 3.772 ... -10.02 -10.08
    old_std   (month, lat, lon) float32 3.856 4.789 4.71 ... 14.88 14.94 15.25

In [13]:
os.getcwd()

'd:\\CMAP'

In [65]:
## save
# old_clim.to_netcdf('CMAP_ncfile_all_variables(mean,clim,anom,std)_old.nc')